In [ ]:
%matplotlib qt 

import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from imblearn.metrics import specificity_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from scipy.stats import iqr
from scipy.stats import t

In [ ]:
# import data

data = pd.read_csv('bm_combat_spectral_changed.csv', index_col=[0])

In [ ]:
# harmonized 
data_comb = data.drop(data.iloc[:, 209:], axis=1)
data_comb

In [ ]:
np.random.seed(0)
random.seed(0)

X = data_log
y = data_log[['center','label']]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    random_state=1,
                                                    stratify=X[['center','group']])

In [ ]:
center_names = ['all','california','finland','iowa','medellin']

X_train_sets = {} 
y_train_sets = {}
X_test_sets = {}
y_test_sets = {}


for name in center_names:
    
    if name == 'all':
        X_train_sets.update({name: X_train.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)})
        X_test_sets.update({name: X_test.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)})
        y_train_sets.update({name: y_train.drop(['center'], axis = 1)})
        y_test_sets.update({name: y_test.drop(['center'], axis = 1)})
    
    else: 
        X_train_set = X_train.loc[X_train['center'] == name]
        X_train_set = X_train_set.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)
        X_train_sets.update({name: X_train_set})

        y_train_set = y_train.loc[X_train['center'] == name]
        y_train_set = y_train_set.drop(['center'], axis=1)
        y_train_sets.update({name: y_train_set})

        X_test_set = X_test.loc[X_test['center'] == name]
        X_test_set = X_test_set.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)
        X_test_sets.update({name: X_test_set})

        y_test_set = y_test.loc[X_test['center'] == name]
        y_test_set = y_test_set.drop(['center'], axis=1)
        y_test_sets.update({name: y_test_set})

In [ ]:
# Initializing Classifiers

clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=1)

clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)

clf3 = DecisionTreeClassifier(random_state=1)

clf4 = SVC(random_state=1)

# Building the pipelines

pipe1 = Pipeline([('clf1', clf1)])

pipe2 = Pipeline([('clf2', clf2)])

pipe4 = Pipeline([('clf4', clf4)])


# Setting up the parameter grids
param_grid1 = [{'clf1__penalty': ['l2'],
                'clf1__C': np.power(10., np.arange(-4, 4))}]

param_grid2 = [{'clf2__n_neighbors': list(range(1, 10)),
                'clf2__p': [1, 2]}]

param_grid3 = [{'max_depth': list(range(1, 10)) + [None],
                'criterion': ['gini', 'entropy']}]


param_grid4 = [
               {'clf4__kernel': ['rbf'],
                'clf4__C': np.power(10., np.arange(-4, 4)),
                'clf4__gamma': np.power(10., np.arange(-5, 0))},
               {'clf4__kernel': ['linear'],
                'clf4__C': np.power(10., np.arange(-4, 4))}
              ]

In [ ]:
# Setting up multiple GridSearchCV objects, 1 for each algorithm


gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2,
                             param_grid3, param_grid4),
                            (pipe1, pipe2, clf3, pipe4),
                            ('Softmax', 'KNN', 'DTree', 'SVM')):
    
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=5,
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

# FEATURE SELECTION METHOD: SELECTKBEST

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
list_of_k = np.linspace(1, 203, num=203, dtype=int)
list_of_k

In [ ]:
metrics = ['accuracy','recall','specificity','precision','F1','auc']
folds = [1, 2, 3, 4, 5]
feature_scoring = {k: {center_name: [] for center_name in center_names} for k in list_of_k}
cv_scores = {k: {center_name: {name: {metric: [] for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
cv_scores_without_zeros = {k: {center_name: {name: {metric: [] for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

features_ids = {k: {center_name: [] for center_name in center_names} for k in list_of_k}
features_names = {k: {center_name: [] for center_name in center_names} for k in list_of_k}

for k in list_of_k:

    for center_name in center_names:
        
        if center_name != center_names[0]: continue

        print('Cross-validation + feature selection for number of features of ' + str(k) + ' for ' + center_name.upper())

        # The outer loop for algorithm selection
        c = 1
        for outer_train_idx, outer_valid_idx in skfold.split(X_train_sets[center_name],y_train_sets[center_name]):

            # feature selection: starts here

            selector = SelectKBest(score_func=f_classif, k=k)

            # run score function on (X,y) and get the appropriate features
            fit = selector.fit(X_train_sets[center_name].iloc[outer_train_idx.tolist()], y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel())
            
            feature_scoring[k][center_name].append(fit.scores_)

            # Get columns to keep and create new dataframe with those only
            selected_features = fit.get_support(indices=True)

            features = X_train_sets[center_name].columns[selected_features].to_list()
            
            features_ids[k][center_name].append(selected_features)
            features_names[k][center_name].append(features)

            # feature selection: ends here

            for name, gs_est in sorted(gridcvs.items()):

                print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

                # The inner loop for hyperparameter tuning

                if len(features) != 0:

                    gs_est.fit(X_train_sets[center_name].iloc[outer_train_idx.tolist()].iloc[:,selected_features], y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel())
                    y_pred = gs_est.predict(X_train_sets[center_name].iloc[outer_valid_idx.tolist()].iloc[:,selected_features])


                    for metric in metrics: 

                        if metric == 'accuracy':
                            calc_metric = accuracy_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                                  (gs_est.best_score_ * 100, calc_metric * 100))
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'recall':
                            calc_metric = recall_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'specificity':
                            calc_metric = specificity_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'precision':
                            calc_metric = precision_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'F1':
                            calc_metric = f1_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        else:
                            calc_metric = roc_auc_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_score=y_pred, average = 'macro')
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                else:

                    print('!!!NO FEATURES WERE SELECTED!!!')

                    gs_est.fit(X_train_sets[center_name].iloc[outer_train_idx.tolist()], y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel())
                    y_pred = gs_est.predict(X_train_sets[center_name].iloc[outer_valid_idx.tolist()])

                    for metric in metrics: 

                        if metric == 'accuracy':
                            calc_metric = accuracy_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                                  (gs_est.best_score_ * 100, calc_metric * 100))
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'recall':
                            calc_metric = recall_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'specificity':
                            calc_metric = specificity_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'precision':
                            calc_metric = precision_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'F1':
                            calc_metric = f1_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        else:
                            calc_metric = roc_auc_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_score=y_pred, average = 'macro')
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)


            c += 1

In [ ]:
# Looking at the results


mean_std_cv_scores = {k: {center_name: {name: {metric: {'mean': None, 'SD': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
mean_std_cv_scores_without_zeros = {k: {center_name: {name: {metric: {'mean': None, 'SD': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
median_iqr_cv_scores = {k: {center_name: {name: {metric: {'median': None, 'IQR': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
median_iqr_cv_scores_without_zeros = {k: {center_name: {name: {metric: {'median': None, 'IQR': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}

for k in list_of_k:
    
    for center_name in center_names:
    
        for name, gs_est in sorted(gridcvs.items()):
            print('Results for ' + center_name.upper() + ' ' + str(k) + ' ' + name.upper())
            for metric in metrics: 
                if metric != 'auc':
                    print('%-8s | outer CV ' + metric + ' %.2f%% +\- %.3f' % (
                      100 * np.mean(cv_scores[k][center_name][name][metric]), 100 * np.std(cv_scores[k][center_name][name][metric])))
                    mean_std_cv_scores[k][center_name][name][metric]['mean'] = np.mean(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['mean'] = np.mean(cv_scores_without_zeros[k][center_name][name][metric])
                    mean_std_cv_scores[k][center_name][name][metric]['SD'] = np.std(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['SD'] = np.std(cv_scores_without_zeros[k][center_name][name][metric])
                    
                    median_iqr_cv_scores[k][center_name][name][metric]['median'] = 100*np.median(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['median'] = 100*np.median(cv_scores_without_zeros[k][center_name][name][metric])
                    median_iqr_cv_scores[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores_without_zeros[k][center_name][name][metric])
                else: 
                    print('%-8s | outer CV ' + metric + ' %.2f +\- %.3f' % (
                      np.mean(cv_scores[k][center_name][name][metric]), np.std(cv_scores[k][center_name][name][metric])))
                    mean_std_cv_scores[k][center_name][name][metric]['mean'] = np.mean(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['mean'] = np.mean(cv_scores_without_zeros[k][center_name][name][metric])
                    mean_std_cv_scores[k][center_name][name][metric]['SD'] = np.std(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['SD'] = np.std(cv_scores_without_zeros[k][center_name][name][metric])
                    
                    median_iqr_cv_scores[k][center_name][name][metric]['median'] = 100*np.median(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['median'] = 100*np.median(cv_scores_without_zeros[k][center_name][name][metric])
                    median_iqr_cv_scores[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores_without_zeros[k][center_name][name][metric])
        print('*********************')
#print('\nSoftmax Best parameters', gridcvs['Softmax'].best_params_)
#print('\nKNN Best parameters', gridcvs['KNN'].best_params_)
#print('\nDTree Best parameters', gridcvs['DTree'].best_params_)
#print('\nSVM Best parameters', gridcvs['SVM'].best_params_)

In [ ]:
confedence_interval = {k: {center_name: {name: {metric: () for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}

dof = 5-1 
confidence = 0.95

for k in list_of_k:
    for center_name in center_names:
        for name, gs_est in sorted(gridcvs.items()):
            for metric in metrics:

                m = mean_std_cv_scores[k][center_name][name][metric]['mean']
                s = mean_std_cv_scores[k][center_name][name][metric]['SD']

                t_crit = np.abs(t.ppf((1-confidence)/2,dof))
                confedence_interval[k][center_name][name][metric]= ((m-s*t_crit/np.sqrt(5))*100, (m+s*t_crit/np.sqrt(5))*100)
            
confedence_interval

In [ ]:
cv_acc_models_onlyinner = {metric: {name: {'mean': [], 'std': []} for name, gs_est in sorted(gridcvs.items())} for metric in metrics}

for k in list_of_k:
    for name, gs_est in sorted(gridcvs.items()):
        for metric in metrics:
            cv_acc_models_onlyinner[metric][name]['mean'].append(cv_scores_onlyinner[k]['all'][name][metric]['mean'])
            cv_acc_models_onlyinner[metric][name]['std'].append(cv_scores_onlyinner[k]['all'][name][metric]['std'])
cv_acc_models

In [ ]:
for name, gs_est in sorted(gridcvs.items()):
    plt.figure()
    #plt.scatter(list_of_k, cv_acc_models['accuracy'][name]['mean'])
    plt.errorbar(x=list_of_k, y=cv_acc_models_onlyinner['accuracy'][name]['mean'], yerr=cv_acc_models_onlyinner['accuracy'][name]['std'], linestyle='None', marker='o')
    plt.xlabel('number of features')
    plt.ylabel('cv accuracy')
    plt.title('cv accuracy for ' + name)
    plt.show()

In [ ]:
merged_features_ids = {k: {center_name: [] for center_name in center_names} for k in list_of_k}
merged_features_names = {k: {center_name: [] for center_name in center_names} for k in list_of_k}

for k in list_of_k:
    
    for center_name in center_names:
    
        #print(center_name)
        for i in range(len(features_ids[k][center_name])):
            #print(i)
            length = len(features_ids[k][center_name][i])
            #print(length)
            for j in range(length):
                merged_features_ids[k][center_name].append(features_ids[k][center_name][i][j])
                merged_features_names[k][center_name].append(features_names[k][center_name][i][j])

merged_features_ids

In [ ]:
for k in list_of_k:
    for center_name in center_names: 
        print('Number of features: ' + str(k))
        print(len(merged_features_ids[k][center_name]))

In [ ]:
import operator

counts = {k: {center_name: {} for center_name in center_names} for k in list_of_k}

for k in list_of_k: 
    print('Number of features: ' + str(k))
    for center_name in center_names:
        print(center_name)
        
        for feature_name in merged_features_names[k][center_name]:
            #print(feature_name)
            count = merged_features_names[k][center_name].count(feature_name)
            #print(count)
            counts[k][center_name].update({feature_name: count})

        counts[k][center_name] = sorted(counts[k][center_name].items(),key=operator.itemgetter(1),reverse=True)
        print(len(counts[k][center_name]))
        
print(counts)

In [ ]:
for k in list_of_k:
    for center_name in center_names:
        print(center_name)
        print('Number of features: ' + str(k))
        merged_features_ids[k][center_name] = list(set(merged_features_ids[k][center_name]))
        print(len(merged_features_ids[k][center_name]))

In [ ]:
# BOOTSTRAPPING

from sklearn.utils import resample

# Fitting a model to the whole training set
# using the "best" algorithm
best_algo = gridcvs['Softmax']


bootstrap = {center_name: {metric: [] for metric in metrics} for center_name in center_names}

for center_name in center_names:
    
    print('Test results for ' + center_name.upper())
    
    #Lets configure Bootstrap
    
    print(X_test_sets[center_name].shape)

    n_iterations = 100  # No. of bootstrap samples to be repeated (created)
    n_size = int(len(X_test_sets[center_name]) * 1.0) # Size of sample, picking only 50% of the given data in every bootstrap sample
    
    for i in range(n_iterations):
    
        test_values = resample(X_test_sets[center_name], n_samples = n_size)
        print(test_values.shape)
        print(test_values.index.to_list())
        test_labels = y_test_sets[center_name].loc[test_values.index.to_list()]


        if center_name == 'all':
            start = time.time()
            best_algo.fit(X_train_sets[center_name].iloc[:,merged_features_ids[67]['all']], y_train_sets[center_name].values.ravel())
            stop = time.time()

            for metric in metrics: 

                if metric == 'accuracy':
                    calc_metric = accuracy_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Test Accuracy: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'recall':
                    calc_metric = recall_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Recall/sensitivity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'specificity':
                    calc_metric = specificity_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Specificity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'precision':
                    calc_metric = precision_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Precision: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'F1':
                    calc_metric = f1_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('F1 score: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                else:
                    calc_metric = roc_auc_score(y_true=test_labels, y_score=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]), average = 'macro')
                    print('ROC AUC: %.2f' % calc_metric)
                    bootstrap[center_name][metric].append(calc_metric)

        else:

            for metric in metrics: 

                if metric == 'accuracy':
                    calc_metric = accuracy_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Test Accuracy: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'recall':
                    calc_metric = recall_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Recall/sensitivity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'specificity':
                    calc_metric = specificity_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Specificity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'precision':
                    calc_metric = precision_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('Precision: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'F1':
                    calc_metric = f1_score(y_true=test_labels, y_pred=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]))
                    print('F1 score: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                else:
                    calc_metric = roc_auc_score(y_true=test_labels, y_score=best_algo.predict(test_values.iloc[:,merged_features_ids[67]['all']]), average = 'macro')
                    print('ROC AUC: %.2f' % calc_metric)
                    bootstrap[center_name][metric].append(calc_metric)

In [ ]:
for center_name in center_names:
    print('Results for ' + center_name.upper())
    for metric in metrics: 
        if metric != 'auc':
            print(metric + ' %.2f%% +\- %.3f' % (
              100 * np.mean(bootstrap[center_name][metric]), 100 * np.std(bootstrap[center_name][metric])))
        else: 
            print(metric + ' %.2f +\- %.3f' % (
              np.mean(bootstrap[center_name][metric]), np.std(bootstrap[center_name][metric])))